# Image Upload and Object Detection with YOLOv8

This notebook demonstrates how to perform object detection on uploaded images using pre-trained YOLOv8 models. The notebook is designed to work in both Google Colab and Kaggle environments.

**Contents:**
1. Setting up the environment
2. Importing the YOLOv8 detector
3. Uploading and preparing images
4. Performing object detection
5. Displaying and analyzing results
6. Batch processing multiple images
7. Advanced options for object detection

## 1. Setting up the Environment

First, we'll set up the environment and install the necessary packages.

In [ ]:
# Clone the repository if running in Colab/Kaggle
import os
import sys

# Check if we're in Colab or Kaggle
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'kaggle_secrets' in sys.modules

# If we're in Colab or Kaggle, set up the environment
if IN_COLAB or IN_KAGGLE:
    # Clone the repository
    !git clone -q https://github.com/yourusername/object-detection-yolo.git
    %cd object-detection-yolo
    
    # Install dependencies
    !pip install -q ultralytics opencv-python ipywidgets matplotlib Pillow
    
    # Add the repository root to the Python path
    sys.path.insert(0, os.getcwd())
    
    print(f"Setting up in {'Google Colab' if IN_COLAB else 'Kaggle'}")
else:
    print("Running locally")

In [ ]:
# Import the utilities for notebook environment
try:
    from utils.notebook_utils import (
        setup_env, show_upload_widget, process_upload_widget_kaggle,
        process_upload_colab, display_image_with_info, download_sample_images
    )
except ImportError:
    # If imports fail, create a minimal version of necessary functions
    def setup_env():
        print("Environment setup simplified due to import error")
        if 'google.colab' in sys.modules:
            !pip install -q ultralytics opencv-python ipywidgets
            from google.colab import files
            print("Running in Google Colab")
        elif 'kaggle_secrets' in sys.modules:
            !pip install -q ultralytics opencv-python ipywidgets
            print("Running in Kaggle")
        else:
            print("Running locally")
    
    def show_upload_widget():
        if 'google.colab' in sys.modules:
            from google.colab import files
            print("Select an image to upload:")
            return files.upload()
        else:
            try:
                import ipywidgets as widgets
                from IPython.display import display
                file_upload = widgets.FileUpload(
                    accept='.jpg,.jpeg,.png',
                    multiple=False,
                    description='Upload:'
                )
                display(file_upload)
                return file_upload
            except ImportError:
                print("Please install ipywidgets for upload functionality")
                return None
    
    def process_upload_widget_kaggle(file_upload):
        if not file_upload.value:
            return []
        
        import os
        os.makedirs('uploads', exist_ok=True)
        
        paths = []
        for name, file_info in file_upload.value.items():
            path = f'uploads/{name}'
            with open(path, 'wb') as f:
                f.write(file_info['content'])
            paths.append(path)
        
        return paths
    
    def process_upload_colab(uploaded_files):
        if not uploaded_files:
            return []
        
        import os
        os.makedirs('uploads', exist_ok=True)
        
        paths = []
        for name, content in uploaded_files.items():
            path = f'uploads/{name}'
            with open(path, 'wb') as f:
                f.write(content)
            paths.append(path)
        
        return paths
    
    def display_image_with_info(image_path):
        import cv2
        import matplotlib.pyplot as plt
        from pathlib import Path
        
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        print(f"Image: {Path(image_path).name}")
        print(f"Resolution: {img.shape[1]}x{img.shape[0]}")
        
        plt.figure(figsize=(10, 8))
        plt.imshow(img_rgb)
        plt.title(Path(image_path).name)
        plt.axis('off')
        plt.show()
    
    def download_sample_images():
        import os
        import urllib.request
        from pathlib import Path
        
        sample_dir = Path('sample_images')
        sample_dir.mkdir(exist_ok=True)
        
        samples = [
            {'url': 'https://ultralytics.com/images/zidane.jpg', 'name': 'person.jpg'},
            {'url': 'https://ultralytics.com/images/bus.jpg', 'name': 'bus.jpg'}
        ]
        
        paths = []
        for sample in samples:
            path = sample_dir / sample['name']
            if not path.exists():
                print(f"Downloading {sample['name']}...")
                urllib.request.urlretrieve(sample['url'], path)
            paths.append(str(path))
        
        return paths

# Set up the environment
setup_env()

## 2. Importing the YOLOv8 Detector

Now, let's import the YOLOv8 detector class that we created earlier.

In [ ]:
# Import YOLOv8 detector
try:
    from src.yolo_detector import YOLOv8Detector
except ImportError:
    # If the import fails, define a minimal version here
    from ultralytics import YOLO
    import torch
    import cv2
    import numpy as np
    import matplotlib.pyplot as plt
    from pathlib import Path
    import time
    
    class YOLOv8Detector:
        def __init__(self, model_size='n', conf=0.25, iou=0.45, device=None):
            if device is None:
                self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            else:
                self.device = device
                
            model_path = f'yolov8{model_size}.pt'
            self.model = YOLO(model_path)
            
            self.conf = conf
            self.iou = iou
            self.class_names = self.model.names
            
            print(f"YOLOv8{model_size} detector initialized on {self.device}")
        
        def detect(self, image_path, show_result=True, return_processed_image=False):
            result = self.model(image_path, conf=self.conf, iou=self.iou)[0]
            
            if show_result:
                im_array = result.plot()
                plt.figure(figsize=(12, 8))
                plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                plt.title(f"Detection Results: {Path(image_path).name}")
                plt.show()
                
                print(f"\nDetections in {Path(image_path).name}:")
                boxes = result.boxes
                for i, box in enumerate(boxes):
                    class_id = int(box.cls.item())
                    class_name = self.class_names[class_id]
                    confidence = box.conf.item()
                    print(f"  {i+1}. {class_name} (Confidence: {confidence:.2f})")
            
            if return_processed_image:
                return result.plot()
            
            return result
        
        def detect_batch(self, images, show_results=True, max_display=10):
            results = self.model(images, conf=self.conf, iou=self.iou)
            
            if show_results:
                display_count = min(len(results), max_display)
                rows = (display_count + 2) // 3
                cols = min(display_count, 3)
                
                plt.figure(figsize=(18, 6 * rows))
                
                for i in range(display_count):
                    plt.subplot(rows, cols, i + 1)
                    im_array = results[i].plot()
                    plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
                    
                    if isinstance(images[i], str):
                        plt.title(f"Detection: {Path(images[i]).name}")
                    else:
                        plt.title(f"Detection #{i+1}")
                    
                    plt.axis('off')
                
                plt.tight_layout()
                plt.show()
            
            return results
        
        def print_model_summary(self):
            print(f"YOLOv8 Model Summary:")
            print(f"Task: {self.model.task}")
            print(f"Number of classes: {len(self.class_names)}")
            print(f"Inference device: {self.device}")

In [ ]:
# Initialize the YOLOv8 detector
detector = YOLOv8Detector(model_size='n', conf=0.25, iou=0.45)

# Print model summary
detector.print_model_summary()

## 3. Uploading and Preparing Images

Let's create a function to upload images for object detection. We'll provide options for both uploading your own images and using sample images.

In [ ]:
def get_images_for_detection():
    """Get images for object detection through upload or samples."""
    import ipywidgets as widgets
    from IPython.display import display
    
    # Create radio buttons for image source selection
    image_source = widgets.RadioButtons(
        options=['Upload my own images', 'Use sample images'],
        description='Image source:',
        disabled=False
    )
    
    display(image_source)
    
    # Wait for selection
    if image_source.value == 'Upload my own images':
        # Show upload widget
        uploaded = show_upload_widget()
        
        # Process uploaded files
        if 'google.colab' in sys.modules:
            image_paths = process_upload_colab(uploaded)
        else:
            # Wait for user to upload files
            print("\nAfter uploading, run the next cell to process the images.")
            return uploaded  # Return the widget for processing in the next cell
    else:
        # Download and use sample images
        image_paths = download_sample_images()
        print(f"\nUsing {len(image_paths)} sample images.")
    
    return image_paths

In [ ]:
# Get images for detection
image_source = get_images_for_detection()

In [ ]:
# Process uploaded images if needed
if not isinstance(image_source, list):
    # This means we got a file upload widget
    image_paths = process_upload_widget_kaggle(image_source)
    print(f"Processed {len(image_paths)} uploaded images.")
else:
    # We already have image paths
    image_paths = image_source

# Display the images
if image_paths:
    for path in image_paths:
        display_image_with_info(path)
else:
    print("No images available for processing.")

## 4. Performing Object Detection

Now that we have our images, let's perform object detection on them using the YOLOv8 detector.

In [ ]:
# Detect objects in each image
if image_paths:
    for path in image_paths:
        print(f"\nProcessing {path}...")
        result = detector.detect(path, show_result=True)
else:
    print("No images available for detection.")

## 5. Displaying and Analyzing Results

Let's create a function to display more detailed analysis of the detection results, including class distributions and detection confidence.

In [ ]:
def analyze_detection_results(image_path, result):
    """Analyze and display detailed detection results."""
    import matplotlib.pyplot as plt
    import numpy as np
    from collections import Counter
    from pathlib import Path
    
    # Get detection boxes
    boxes = result.boxes
    
    if len(boxes) == 0:
        print(f"No objects detected in {Path(image_path).name}")
        return
    
    # Get class IDs and confidences
    class_ids = [int(box.cls.item()) for box in boxes]
    confidences = [box.conf.item() for box in boxes]
    class_names = [detector.class_names[class_id] for class_id in class_ids]
    
    # Count objects by class
    class_counts = Counter(class_names)
    
    # Create figure for analysis
    plt.figure(figsize=(18, 10))
    
    # Plot the detection image
    plt.subplot(2, 2, 1)
    im_array = result.plot()
    plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
    plt.title(f"Detection Results: {Path(image_path).name}")
    plt.axis('off')
    
    # Plot class distribution
    plt.subplot(2, 2, 2)
    classes, counts = zip(*class_counts.items()) if class_counts else ([], [])
    y_pos = np.arange(len(classes))
    plt.barh(y_pos, counts, align='center')
    plt.yticks(y_pos, classes)
    plt.xlabel('Count')
    plt.title('Object Classes')
    
    # Plot confidence distribution
    plt.subplot(2, 2, 3)
    plt.hist(confidences, bins=10, range=(0, 1))
    plt.xlabel('Confidence')
    plt.ylabel('Count')
    plt.title('Confidence Distribution')
    
    # Plot box sizes
    if len(boxes) > 0:
        plt.subplot(2, 2, 4)
        box_areas = []
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            box_areas.append((x2 - x1) * (y2 - y1))
        
        plt.scatter(range(len(box_areas)), box_areas)
        plt.xlabel('Box Index')
        plt.ylabel('Box Area (pixels²)')
        plt.title('Bounding Box Sizes')
    
    plt.tight_layout()
    plt.show()
    
    # Print detection summary
    print(f"\nDetection Summary for {Path(image_path).name}:")
    print(f"Total objects detected: {len(boxes)}")
    print("\nObjects by class:")
    for class_name, count in class_counts.items():
        print(f"  {class_name}: {count}")
    
    print(f"\nConfidence range: {min(confidences):.2f} - {max(confidences):.2f}")
    print(f"Average confidence: {sum(confidences)/len(confidences):.2f}")

In [ ]:
# Analyze detection results
if image_paths:
    for path in image_paths:
        # Run detection again to get results
        result = detector.detect(path, show_result=False)
        
        # Analyze the results
        analyze_detection_results(path, result)
else:
    print("No images available for analysis.")

## 6. Batch Processing Multiple Images

For efficiency, we can process multiple images in a batch. This is useful when you have many images to process.

In [ ]:
# Batch process all images
if len(image_paths) > 1:
    print(f"Batch processing {len(image_paths)} images...")
    batch_results = detector.detect_batch(image_paths, show_results=True)
    print(f"Batch processing complete.")
else:
    print("Need multiple images for batch processing.")

## 7. Advanced Options for Object Detection

Let's create an interactive interface to adjust detection parameters and see how they affect the results.

In [ ]:
def interactive_detection(image_path):
    """Create an interactive interface for object detection."""
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    
    # Create widgets for parameters
    model_size = widgets.Dropdown(
        options=['n', 's', 'm', 'l', 'x'],
        value='n',
        description='Model Size:',
        disabled=False,
    )
    
    conf_threshold = widgets.FloatSlider(
        value=0.25,
        min=0.01,
        max=0.99,
        step=0.05,
        description='Confidence:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
    )
    
    iou_threshold = widgets.FloatSlider(
        value=0.45,
        min=0.1,
        max=0.9,
        step=0.05,
        description='IoU:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
    )
    
    # Create a button to run detection
    run_button = widgets.Button(
        description='Run Detection',
        disabled=False,
        button_style='success',
        tooltip='Click to run detection with selected parameters',
        icon='check'
    )
    
    # Output widget to display results
    output = widgets.Output()
    
    # Function to run when button is clicked
    def on_button_clicked(b):
        with output:
            clear_output()
            print(f"Running detection with:")
            print(f"Model: YOLOv8{model_size.value}")
            print(f"Confidence threshold: {conf_threshold.value}")
            print(f"IoU threshold: {iou_threshold.value}")
            
            # Initialize detector with selected parameters
            detector = YOLOv8Detector(
                model_size=model_size.value,
                conf=conf_threshold.value,
                iou=iou_threshold.value
            )
            
            # Run detection
            result = detector.detect(image_path, show_result=True)
            
            # Show detailed analysis
            analyze_detection_results(image_path, result)
    
    # Connect the button to the function
    run_button.on_click(on_button_clicked)
    
    # Display the widgets
    print(f"Interactive Detection for {Path(image_path).name}")
    print("Select parameters and click 'Run Detection' to see results.")
    display(widgets.VBox([
        widgets.HBox([model_size, conf_threshold, iou_threshold]),
        run_button
    ]))
    display(output)

In [ ]:
# Run interactive detection on the first image
if image_paths:
    interactive_detection(image_paths[0])
else:
    print("No images available for interactive detection.")

## 8. Saving Detection Results

Let's create a function to save the detection results to disk, which can be useful for further analysis or sharing.

In [ ]:
def save_detection_results(image_paths, output_dir='detection_results'):
    """Save detection results to disk."""
    import os
    from pathlib import Path
    import cv2
    import json
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each image
    results = []
    for i, path in enumerate(image_paths):
        print(f"Processing image {i+1}/{len(image_paths)}: {Path(path).name}")
        
        # Run detection
        result = detector.detect(path, show_result=False)
        
        # Get detection image
        detection_img = result.plot()
        
        # Save detection image
        output_path = os.path.join(output_dir, f"detection_{Path(path).stem}.jpg")
        cv2.imwrite(output_path, detection_img)
        
        # Prepare detection data for JSON
        boxes = result.boxes
        detection_data = {
            'image_name': Path(path).name,
            'detections': []
        }
        
        for box in boxes:
            class_id = int(box.cls.item())
            class_name = detector.class_names[class_id]
            confidence = float(box.conf.item())
            bbox = [float(x) for x in box.xyxy[0].tolist()]  # xyxy format is [x1, y1, x2, y2]
            
            detection_data['detections'].append({
                'class_id': class_id,
                'class_name': class_name,
                'confidence': confidence,
                'bbox': bbox
            })
        
        # Save detection data to JSON
        json_path = os.path.join(output_dir, f"detection_{Path(path).stem}.json")
        with open(json_path, 'w') as f:
            json.dump(detection_data, f, indent=2)
        
        results.append({
            'image_path': path,
            'detection_image': output_path,
            'detection_data': json_path,
            'num_detections': len(boxes)
        })
    
    print(f"\nSaved detection results for {len(image_paths)} images to {output_dir}/")
    
    # If in Colab, provide a download link
    if 'google.colab' in sys.modules:
        from google.colab import files
        
        # Zip the results directory
        !zip -r {output_dir}.zip {output_dir}
        
        # Download the zip file
        print("\nDownloading results as a zip file...")
        files.download(f"{output_dir}.zip")
    
    return results

In [ ]:
# Save detection results
if image_paths:
    saved_results = save_detection_results(image_paths)
    
    # Display summary
    print("\nDetection Results Summary:")
    for result in saved_results:
        print(f"  {Path(result['image_path']).name}: {result['num_detections']} objects detected")
else:
    print("No images available for saving results.")

## 9. Summary and Next Steps

In this notebook, we've demonstrated how to:

1. Set up the environment for object detection
2. Import and configure the YOLOv8 detector
3. Upload and prepare images for detection
4. Perform object detection on individual images
5. Analyze and visualize detection results
6. Batch process multiple images
7. Use interactive controls to adjust detection parameters
8. Save detection results to disk

### Next Steps

In the next notebook, we'll explore how to perform real-time object detection using a webcam or video feed. This will allow you to detect objects in a live video stream, which is useful for applications like surveillance, robotics, and interactive systems.